1. 데이터수집

In [5]:
import codecs
with codecs.open('data2.txt',encoding='utf-8') as f: #문서 로드
    #한줄의 문장이 id, docment label이 '\t'로 분리되어있다.
    data = [문장.split('\t') for 문장 in f.read().splitlines()]
    data=data[1:] #첫번째 feature_names제거



2-1. 데이터전처리(문장 토큰화 및 정제)

In [7]:
t_data = [문장[1] for 문장 in data]
#우리는 document 피쳐만이 필요하다.

2-2. 데이터전처리(단어토큰화->형태소 토큰화)

In [13]:
from konlpy.tag import Okt
from tqdm import tqdm #작업의 진행사항을 알려주는 패키지
from nltk.util import ngrams #ngrams 
tk = Okt()
def tk_f(t_data):
    tk_d = ['/'.join(x) for x in tk.pos(t_data)] # 각 줄을 형태소로 분리하고'/'로 한줄을 만든다.
    return tk_d
end_data=[]

for t in tqdm(t_data): #반복문을 통한 작업 진행사항 확인
    tk_data = tk_f(t) #/'+형택소로 만들어진 데이터#문자열 데이터 #문장->형태소 문장
    #바이그램을 만들기 위한 SS<문장>SE전처리
    bg=ngrams(tk_data,2,pad_left=True,pad_right=True,left_pad_symbol="SS",right_pad_symbol='SE')#문장->바이그램화
    end_data+=[t for t in bg] #바이그램 각 토큰을 end_data에 삽입

    

100%|████████████████████████████████████████████████████████████████████| 150000/150000 [21:28<00:00, 116.38it/s]


AttributeError: 'list' object has no attribute 'shape'

In [15]:
from nltk import ConditionalFreqDist  #문맥별 빈도수 계산 클래스
from nltk.probability import ConditionalProbDist,MLEProbDist #문맥별 조건부 확률 딕셔너리

In [16]:
end_data[0:10]

[('SS', '아/Exclamation'),
 ('아/Exclamation', '더빙/Noun'),
 ('더빙/Noun', '../Punctuation'),
 ('../Punctuation', '진짜/Noun'),
 ('진짜/Noun', '짜증나네요/Adjective'),
 ('짜증나네요/Adjective', '목소리/Noun'),
 ('목소리/Noun', 'SE'),
 ('SS', '흠/Noun'),
 ('흠/Noun', '.../Punctuation'),
 ('.../Punctuation', '포스터/Noun')]

In [17]:
cfd = ConditionalFreqDist(end_data)
cpd = ConditionalProbDist(cfd, MLEProbDist)

In [18]:
st='SS'
all_str=[]
import random
random.seed(0)
while True: 
    st=cpd[st].generate() #'st'다음 단어 랜덤 생성
    if st=='SE':
        break
    d=st.split("/")[0] #단, 형태소의 는 버리고/
    all_str.append(d) #출력할 형태소 리스트에 추가
n_data=''.join(all_str)
n_data

'미키짱과말도전혀빗나가지않던전개로꽥꽥대는거보니까요^^'

In [19]:
def 정리_생성():
    c = "SS"
    sentence = []
    while True:
        w = cpd[c].generate()

        if w == "SE":
            break

        w2 = w.split("/")[0]
        pos = w.split("/")[1]

        if c == "SS":
            sentence.append(w2.title())
        elif c in ["`", "\"", "'", "("]:
            sentence.append(w2)
        elif w2 in ["'", ".", ",", ")", ":", ";", "?"]:
            sentence.append(w2)
        elif pos in ["Josa", "Punctuation", "Suffix"]:
            sentence.append(w2)
        elif w in ["임/Noun", "것/Noun", "는걸/Noun", "릴때/Noun",
                   "되다/Verb", "이다/Verb", "하다/Verb", "이다/Adjective"]:
            sentence.append(w2)
        else:
            sentence.append(" " + w2)
        c = w

    return "".join(sentence)

In [20]:
정리_생성()

'빨강 머리 박고 싸우는 장면의 호러-> 1 점 만점이다라고 한번은 분명히 보는 편 빼고는 다 스스로 인식에서 폭풍 오열 하는 프로는 장면 속이 시원했어요 주인공이 산만했다 그래도 재밌지 않나? 이 전설의 뜻'